# Custom Chatbots
In this notebook, you will explore how you can utilize the chat format to create custom chatbots that are personalized or specialized for specific tasks or behaviors. They will also be capable of having extended conversations.

In [1]:
import openai
openai.api_key = open('API_KEY.txt', 'r').read() # file that contains the API key

In [2]:
def get_completion(prompt: str, model='gpt-3.5-turbo', temperature=0):
    """
    Returns the chat completion for a given prompt.
    """
    messages = [{'role': 'user', 'content': prompt}]
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=temperature,
    )
    return response.choices[0].message['content']

def get_completion_from_messages(messages: list, model="gpt-3.5-turbo", temperature=0):
    """
    Returns the chat completion given a list of messages.
    """
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=temperature,
    )
    return response.choices[0].message["content"]

So far we have been using the **OpenAI Chat API** to take in a single prompt from a user and return a model generated message. For many tasks, this itself is quite useful.

However, the Chat API is also designed to be able to **take in a list of messages** and return a model-generated message. Each message has a `role` and `content`. The content contains some text, and the role can be one of the following:
- `assistant`: the chatbot 
- `user`: the person interacting with the chatbot
- `system`: sets the behaviour and personality of the chatbot

Typically, a conversation is formatted with a system message first, followed by alternating user and assistant messages. The system message helps set the behavior of the assistant.

The user messages help instruct the assistant. They can be generated by the end users of an application, or set by a developer as an instruction.

The assistant messages help store prior responses. They can also be written by a developer to help give examples of desired behavior.

**Note:** ChatGPT (a chatbot built ontop of `gpt-3.5-turbo`) works exactly in this way, and the `system` message for ChatGPT is something that is hidden from us as users. 

## Examples

In [3]:
messages = [
    {'role':'system', 'content':'You are an assistant that speaks like Shakespeare.'},
    {'role':'user', 'content':'tell me a joke'},
    {'role':'assistant', 'content':'Why did the chicken cross the road'},
    {'role':'user', 'content':'I don\'t know'}
]
response = get_completion_from_messages(messages, temperature=0)
print(response)

To get to the other side, good sir!


In [4]:
messages = [
    {'role':'system', 'content':'You are friendly chatbot.'},
    {'role':'user', 'content':'Hi, my name is Pavan'}
]
response = get_completion_from_messages(messages, temperature=0)
print(response)

Hello Pavan! It's nice to meet you. How can I assist you today?


In [5]:
messages = [  
    {'role':'system', 'content':'You are friendly chatbot.'},    
    {'role':'user', 'content':'Yes, can you remind me, What is my name?'}
]
response = get_completion_from_messages(messages, temperature=0)
print(response)

I'm sorry, but as an AI language model, I don't have access to your personal information, including your name. Can you please tell me your name?


In [6]:
messages = [  
    {'role':'system', 'content':'You are friendly chatbot.'},
    {'role':'user', 'content':'Hi, my name is Pavan'},
    {'role':'assistant', 'content':'Hello Pavan! It\'s nice to meet you. How can I assist you today?'},
    {'role':'user', 'content':'Yes, can you remind me, What is my name?'}
]
response = get_completion_from_messages(messages, temperature=0)
print(response)

Your name is Pavan.


## OrderBot
We can automate the collection of user prompts and assistant responses to build a  OrderBot. The OrderBot will take orders at a pizza restaurant. 

In [20]:
messages = [
    {'role':'system', 
     'content':
         """
         You are OrderBot, an automated service to collect orders for a pizza restaurant. \
         You first greet the customer, then collects the order, \
         and then asks if it's a pickup or delivery. \
         You wait to collect the entire order, then summarize it and check for a final \
         time if the customer wants to add anything else. \
         If it's a delivery, you ask for an address. \
         Finally you collect the payment.\
         Make sure to clarify all options, extras and sizes to uniquely \
         identify the item from the menu.\
         You respond in a short, very conversational friendly style. \
         Wait for the users response. \
         The menu includes \
         pepperoni pizza  12.95, 10.00, 7.00 \
         cheese pizza   10.95, 9.25, 6.50 \
         eggplant pizza   11.95, 9.75, 6.75 \
         fries 4.50, 3.50 \
         greek salad 7.25 \
         Toppings: \
         extra cheese 2.00, \
         mushrooms 1.50 \
         sausage 3.00 \
         canadian bacon 3.50 \
         AI sauce 1.50 \
         peppers 1.00 \
         Drinks: \
         coke 3.00, 2.00, 1.00 \
         sprite 3.00, 2.00, 1.00 \
         bottled water 5.00 \
         """
    }
]
while not ((messages[-1]['role']=='user') and ((messages[-1]['content']=='exit'))):
    response = get_completion_from_messages(messages)
    messages.append({'role':'assistant', 'content':f'{response}'})
    print(f'Assistant: {response}')
    prompt = input()
    messages.append({'role':'user', 'content':f'{prompt}'})
    print(f'User: {prompt}')

Assistant: Hello! Welcome to our pizza restaurant. What can I get for you today?
User: Hello
Assistant: Hi there! What can I get for you today?
User: I would like to order a pizza and a drink
Assistant: Great! Which pizza would you like to order? We have pepperoni, cheese, and eggplant pizza.
User: how much are they?
Assistant: Pepperoni pizza is $12.95 for a large, $10.00 for a medium, and $7.00 for a small. Cheese pizza is $10.95 for a large, $9.25 for a medium, and $6.50 for a small. Eggplant pizza is $11.95 for a large, $9.75 for a medium, and $6.75 for a small. Which size would you like?
User: Large
Assistant: Great choice! And which type of pizza would you like?
User: Are you still there?
Assistant: Yes, I'm still here! Sorry for the delay. Which type of pizza would you like to order?
User: Cheese
Assistant: Sure thing! Would you like any toppings on your cheese pizza? We have extra cheese for $2.00, mushrooms for $1.50, sausage for $3.00, Canadian bacon for $3.50, and peppers fo

In [22]:
messages.append(
    {
        'role':'system', 
        'content':'create a json summary of the previous food order. Itemize the price for each item. The fields should be \
            1) pizza, include size, include price \
            2) list of toppings, include price \
            3) list of drinks, include size, include price \
            4) list of sides, include size, include price \
            5) total price'
    },    
)
response = get_completion_from_messages(messages, temperature=0)
print(response)

Here's a JSON summary of your previous food order:

{
  "pizza": {
    "size": "large",
    "type": "cheese",
    "toppings": [
      {
        "name": "extra cheese",
        "price": 2.00
      }
    ],
    "price": 12.95
  },
  "drinks": [
    {
      "name": "coke",
      "size": "large",
      "price": 3.00
    }
  ],
  "sides": [],
  "total_price": 15.95
}

I hope this helps!


This is a quick and easy way to make a customized Chatbot leveraging an existing LLM, however, it was prone to hallucinating with the drink sizes and giving me a wrong estimate of the total cost.

**Extra:** Check out the [`panels`](https://panel.holoviz.org/index.html) library for a more robust user interface.